# 📘 Step 3: MLflow Tracing - 수동 커스텀 Span

## 목표
- 커스텀 Span으로 세밀한 추적
- RAG 파이프라인의 각 단계별 성능 측정
- 메타데이터 및 태그 추가

## 학습 포인트
- `mlflow.start_span()` 사용법
- Span Attributes vs Tags
- 중첩된 Span 구조 설계
- 디버깅 워크플로우
- 성능 병목 지점 분석

## 0. 사전 준비

### ⚠️ 주의사항
이 노트북을 실행하기 전에:
- **Step 0, 1, 2** 완료 권장
- `.env` 파일 설정 완료
- 필요한 패키지 설치 완료

## 1. 환경 설정 및 패키지 임포트

### 1.1 환경 변수 로드

In [2]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

print("✅ 환경 변수 로드 완료")

✅ 환경 변수 로드 완료


### 1.2 필요한 라이브러리 임포트

In [18]:
import time
import json
from typing import TypedDict, List, Dict, Any
from datetime import datetime

# MLflow
import mlflow
from mlflow import MlflowClient
from mlflow.entities import SpanType, Document, SpanEvent

# LangChain & LangGraph
from langchain_aws import ChatBedrock, BedrockEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate

print("✅ 라이브러리 임포트 완료")
print(f"MLflow 버전: {mlflow.__version__}")

✅ 라이브러리 임포트 완료
MLflow 버전: 3.7.0


## 2. MLflow Tracking 설정

### 2.1 Tracking URI 및 Experiment 설정

In [4]:
# Tracking URI 설정
mlflow.set_tracking_uri("./mlruns")

# Experiment 설정
experiment_name = "rag_agent_custom_spans"
mlflow.set_experiment(experiment_name)

experiment = mlflow.get_experiment_by_name(experiment_name)

print("✅ MLflow 설정 완료")
print(f"📊 Experiment Name: {experiment.name}")
print(f"🆔 Experiment ID: {experiment.experiment_id}")

✅ MLflow 설정 완료
📊 Experiment Name: rag_agent_custom_spans
🆔 Experiment ID: 568044945465180721


/Users/hank_kim/Documents/taehan/mlflow-genai-tutorial/.venv/lib/python3.13/site-packages/mlflow/tracking/_tracking_service/utils.py:177: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance.
  return FileStore(store_uri, store_uri)


## 3. 샘플 데이터 및 Vector Store 준비

### 3.1 샘플 문서 준비

In [5]:
sample_documents = [
    """
    RAG (Retrieval-Augmented Generation)는 대규모 언어 모델(LLM)의 능력을 향상시키는 기술입니다.
    RAG는 외부 지식 베이스에서 관련 정보를 검색하고, 이를 바탕으로 더 정확하고 사실에 기반한 답변을 생성합니다.
    이 방법은 모델이 학습하지 않은 최신 정보나 특정 도메인 지식을 활용할 수 있게 해줍니다.
    """,
    """
    Vector Store는 문서의 임베딩 벡터를 저장하고 검색하는 데이터베이스입니다.
    FAISS, Chroma, Pinecone 등이 대표적인 Vector Store입니다.
    Vector Store는 유사도 검색(Similarity Search)을 통해 쿼리와 가장 관련성 높은 문서를 빠르게 찾아냅니다.
    """,
    """
    Span은 Trace의 구성 요소로, 단일 작업 또는 함수 호출을 나타냅니다.
    Parent Span과 Child Span의 계층 구조를 통해 복잡한 워크플로우를 시각화할 수 있습니다.
    각 Span은 시작 시간, 종료 시간, 입력, 출력, 메타데이터를 포함합니다.
    """,
    """
    커스텀 Span을 사용하면 애플리케이션의 특정 부분을 세밀하게 추적할 수 있습니다.
    mlflow.start_span() 컨텍스트 매니저를 사용하여 수동으로 Span을 생성할 수 있습니다.
    이를 통해 성능 병목 지점을 식별하고 최적화할 수 있습니다.
    """,
]

print(f"✅ {len(sample_documents)}개의 샘플 문서 준비 완료")

✅ 4개의 샘플 문서 준비 완료


### 3.2 Vector Store 생성

In [6]:
# AWS 환경 변수 로드
AWS_MODEL_ID = os.getenv("AWS_MODEL_ID")
AWS_EMD_MODEL_ID = os.getenv("AWS_EMD_MODEL_ID")
AWS_REGION = os.getenv("AWS_REGION", "us-east-1")

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=50,
)

documents = [Document(page_content=doc.strip()) for doc in sample_documents]
split_docs = text_splitter.split_documents(documents)

# 임베딩 및 Vector Store 생성
embeddings = BedrockEmbeddings(
    model_id=AWS_EMD_MODEL_ID,
    region_name=AWS_REGION
)

vector_store = FAISS.from_documents(split_docs, embeddings)

print(f"✅ Vector Store 생성 완료 ({len(split_docs)}개 chunks)")
print(f"📦 임베딩 모델: {AWS_EMD_MODEL_ID}")

✅ Vector Store 생성 완료 (4개 chunks)
📦 임베딩 모델: amazon.titan-embed-text-v2:0


## 4. 커스텀 Span을 사용한 RAG 파이프라인

### 4.1 기본 커스텀 Span 사용

In [7]:
def rag_pipeline_with_custom_spans(query: str, top_k: int = 3) -> Dict[str, Any]:
    """
    커스텀 Span을 사용한 RAG 파이프라인
    """
    print("\n" + "="*80)
    print(f"🔍 Query: {query}")
    print("="*80)
    
    # Root Span 시작
    with mlflow.start_span(name="rag_pipeline", span_type=SpanType.CHAIN) as root_span:
        # Root Span에 입력 설정
        root_span.set_inputs({"query": query, "top_k": top_k})
        
        # Root Span에 메타데이터 추가
        root_span.set_attributes({
            "pipeline_version": "1.0",
            "environment": "development",
            "user_id": "demo_user"
        })
        
        print("\n🌳 Root Span 생성: rag_pipeline")
        
        # Step 1: Document Retrieval
        with mlflow.start_span(name="document_retrieval", span_type=SpanType.RETRIEVER) as retrieval_span:
            print("  ├─ 🔍 Retrieval Span 시작")
            
            retrieval_span.set_inputs({"query": query, "top_k": top_k})
            retrieval_span.set_attributes({
                "retrieval_method": "semantic_search",
                "vector_store_type": "FAISS",
                "embedding_model": AWS_EMD_MODEL_ID,
            })
            
            # 검색 실행
            start_time = time.time()
            retrieved_docs = vector_store.similarity_search(query, k=top_k)
            retrieval_time = time.time() - start_time
            
            # RETRIEVER span 출력 형식 (Document 엔티티 사용)
            retrieval_outputs = [
                Document(
                    page_content=doc.page_content,
                    metadata={"doc_uri": f"doc_{i}", "chunk_id": f"chunk_{i}"}
                )
                for i, doc in enumerate(retrieved_docs)
            ]
            
            retrieval_span.set_outputs(retrieval_outputs)
            retrieval_span.set_attributes({
                "num_docs_retrieved": len(retrieved_docs),
                "retrieval_time_ms": retrieval_time * 1000
            })
            
            print(f"  │  ✅ {len(retrieved_docs)}개 문서 검색 완료 ({retrieval_time:.3f}초)")
        
        # Step 2: Context Preparation
        with mlflow.start_span(name="context_preparation", span_type="PREPROCESSING") as context_span:
            print("  ├─ 📝 Context Preparation Span 시작")
            
            context_span.set_inputs({"documents": [doc.page_content for doc in retrieved_docs]})
            
            # Context 생성
            start_time = time.time()
            context = "\n\n".join([
                f"[문서 {i+1}]\n{doc.page_content}"
                for i, doc in enumerate(retrieved_docs)
            ])
            context_time = time.time() - start_time
            
            context_span.set_outputs({"context": context})
            context_span.set_attributes({
                "context_length": len(context),
                "num_documents": len(retrieved_docs),
                "preparation_time_ms": context_time * 1000
            })
            
            print(f"  │  ✅ Context 준비 완료 ({len(context)} 문자)")
        
        # Step 3: Prompt Construction
        with mlflow.start_span(name="prompt_construction", span_type="PREPROCESSING") as prompt_span:
            print("  ├─ 🔨 Prompt Construction Span 시작")
            
            prompt_template = ChatPromptTemplate.from_messages([
                ("system", "당신은 질문에 답변하는 친절한 AI 어시스턴트입니다. 제공된 컨텍스트를 기반으로 정확하고 간결한 답변을 제공하세요."),
                ("human", "컨텍스트:\n{context}\n\n질문: {query}\n\n답변:")
            ])
            
            messages = prompt_template.format_messages(context=context, query=query)
            prompt_text = "\n".join([msg.content for msg in messages])
            
            prompt_span.set_inputs({"context": context, "query": query})
            prompt_span.set_outputs({"prompt": prompt_text})
            prompt_span.set_attributes({
                "prompt_length": len(prompt_text),
                "template_type": "chat"
            })
            
            print(f"  │  ✅ Prompt 생성 완료 ({len(prompt_text)} 문자)")
        
        # Step 4: Answer Generation
        with mlflow.start_span(name="answer_generation", span_type=SpanType.LLM) as generation_span:
            print("  └─ ✍️  Generation Span 시작")
            
            generation_span.set_inputs({"prompt": prompt_text})
            generation_span.set_attributes({
                "llm_model": AWS_MODEL_ID,
                "temperature": 0.7,
                "max_tokens": 500
            })
            
            # LLM 호출 (AWS Bedrock)
            llm = ChatBedrock(
                model_id=AWS_MODEL_ID,
                region_name=AWS_REGION,
                model_kwargs={"temperature": 0.7}
            )
            
            start_time = time.time()
            response = llm.invoke(messages)
            generation_time = time.time() - start_time
            
            answer = response.content
            
            generation_span.set_outputs({"answer": answer})
            generation_span.set_attributes({
                "answer_length": len(answer),
                "generation_time_ms": generation_time * 1000
            })
            
            print(f"     ✅ 답변 생성 완료 ({generation_time:.3f}초)")
        
        # Root Span 출력 설정
        result = {
            "query": query,
            "answer": answer,
            "num_docs_retrieved": len(retrieved_docs),
            "retrieval_time": retrieval_time,
            "generation_time": generation_time
        }
        
        root_span.set_outputs(result)
        
        print("\n✅ RAG 파이프라인 완료")
        print("="*80)
        
        return result

print("✅ rag_pipeline_with_custom_spans 함수 정의 완료")

✅ rag_pipeline_with_custom_spans 함수 정의 완료


### 4.2 커스텀 Span 실행 테스트

In [8]:
# 테스트 쿼리
test_query = "커스텀 Span이 무엇이고 왜 유용한가요?"

# 파이프라인 실행
result = rag_pipeline_with_custom_spans(test_query, top_k=3)

# 결과 출력
print("\n" + "="*80)
print("📋 실행 결과")
print("="*80)
print(f"\n❓ 질문: {result['query']}")
print(f"\n💬 답변:\n{result['answer']}")
print(f"\n📊 메트릭:")
print(f"  • 검색 시간: {result['retrieval_time']:.3f}초")
print(f"  • 생성 시간: {result['generation_time']:.3f}초")
print(f"  • 검색 문서: {result['num_docs_retrieved']}개")
print("="*80)


🔍 Query: 커스텀 Span이 무엇이고 왜 유용한가요?

🌳 Root Span 생성: rag_pipeline
  ├─ 🔍 Retrieval Span 시작
  │  ✅ 3개 문서 검색 완료 (0.084초)
  ├─ 📝 Context Preparation Span 시작
  │  ✅ Context 준비 완료 (501 문자)
  ├─ 🔨 Prompt Construction Span 시작
  │  ✅ Prompt 생성 완료 (606 문자)
  └─ ✍️  Generation Span 시작
     ✅ 답변 생성 완료 (5.542초)

✅ RAG 파이프라인 완료

📋 실행 결과

❓ 질문: 커스텀 Span이 무엇이고 왜 유용한가요?

💬 답변:
커스텀 Span은 애플리케이션의 특정 부분을 세밀하게 추적하기 위해 수동으로 생성하는 Span입니다.

**커스텀 Span의 특징:**
- `mlflow.start_span()` 컨텍스트 매니저를 사용하여 수동으로 생성
- Trace의 구성 요소로서 단일 작업 또는 함수 호출을 나타냄
- 시작 시간, 종료 시간, 입력, 출력, 메타데이터를 포함

**유용한 이유:**
1. **세밀한 추적**: 애플리케이션의 특정 부분을 상세하게 모니터링
2. **성능 최적화**: 병목 지점을 식별하고 최적화 가능
3. **워크플로우 시각화**: Parent-Child 계층 구조를 통해 복잡한 워크플로우를 시각적으로 이해

커스텀 Span을 활용하면 애플리케이션의 실행 흐름과 성능을 더 깊이 있게 분석할 수 있습니다.

📊 메트릭:
  • 검색 시간: 0.084초
  • 생성 시간: 5.542초
  • 검색 문서: 3개


## 5. Trace Tags 및 Attributes 활용

### 5.1 Trace 레벨 Tags 추가

In [9]:
def rag_pipeline_with_tags(query: str, user_id: str, session_id: str) -> Dict[str, Any]:
    """
    Trace 레벨 태그를 포함한 RAG 파이프라인
    """
    with mlflow.start_span(name="rag_pipeline", span_type=SpanType.CHAIN) as root_span:
        # Trace ID 가져오기
        trace_id = root_span.request_id
        
        # Trace 레벨 태그 설정
        mlflow.set_trace_tag(trace_id, "user_id", user_id)
        mlflow.set_trace_tag(trace_id, "session_id", session_id)
        mlflow.set_trace_tag(trace_id, "environment", "production")
        mlflow.set_trace_tag(trace_id, "user_segment", "premium")
        
        root_span.set_inputs({"query": query})
        
        # 검색
        with mlflow.start_span(name="retrieval", span_type=SpanType.RETRIEVER) as retrieval_span:
            docs = vector_store.similarity_search(query, k=3)
            
            retrieval_outputs = [
                Document(page_content=doc.page_content, metadata={"doc_uri": f"doc_{i}"})
                for i, doc in enumerate(docs)
            ]
            retrieval_span.set_outputs(retrieval_outputs)
        
        # 생성
        with mlflow.start_span(name="generation", span_type=SpanType.LLM) as gen_span:
            context = "\n".join([doc.page_content for doc in docs])
            
            # LLM 호출 (AWS Bedrock)
            llm = ChatBedrock(
                model_id=AWS_MODEL_ID,
                region_name=AWS_REGION,
                model_kwargs={"temperature": 0.7}
            )
            prompt = f"컨텍스트: {context}\n\n질문: {query}\n\n답변:"
            
            response = llm.invoke(prompt)
            answer = response.content
            
            gen_span.set_outputs({"answer": answer})
        
        result = {"query": query, "answer": answer}
        root_span.set_outputs(result)
        
        return result

# 태그와 함께 실행
result_with_tags = rag_pipeline_with_tags(
    query="RAG는 무엇인가요?",
    user_id="user_12345",
    session_id="session_abc"
)

print("✅ Tags가 포함된 Trace 생성 완료")
print(f"💬 답변: {result_with_tags['answer'][:100]}...")

✅ Tags가 포함된 Trace 생성 완료
💬 답변: RAG(Retrieval-Augmented Generation)는 **대규모 언어 모델(LLM)의 능력을 향상시키는 기술**입니다.

주요 특징은 다음과 같습니다:

1. **외부...


### 5.2 Attributes vs Tags 비교

In [10]:
print("\n" + "="*80)
print("📚 Attributes vs Tags 비교")
print("="*80)

comparison = [
    ["특징", "Attributes", "Tags"],
    ["-" * 20, "-" * 35, "-" * 35],
    ["범위", "Span 레벨", "Trace 레벨"],
    ["설정 방법", "span.set_attributes()", "mlflow.set_trace_tag()"],
    ["용도", "실행 메타데이터, 설정값", "사용자/세션 정보, 카테고리"],
    ["변경 가능", "No (불변)", "Yes (UI에서 수정 가능)"],
    ["검색", "Span 필터링", "Trace 필터링"],
    ["예시", "model_name, temperature", "user_id, environment"],
]

for row in comparison:
    print(f"{row[0]:<20} | {row[1]:<35} | {row[2]:<35}")

print("\n" + "="*80)
print("💡 권장 사용법:")
print("  • Attributes: 함수 실행과 직접 관련된 메타데이터")
print("  • Tags: 비즈니스 컨텍스트, 사용자 정보, 환경 정보")
print("="*80)


📚 Attributes vs Tags 비교
특징                   | Attributes                          | Tags                               
-------------------- | ----------------------------------- | -----------------------------------
범위                   | Span 레벨                             | Trace 레벨                           
설정 방법                | span.set_attributes()               | mlflow.set_trace_tag()             
용도                   | 실행 메타데이터, 설정값                       | 사용자/세션 정보, 카테고리                    
변경 가능                | No (불변)                             | Yes (UI에서 수정 가능)                   
검색                   | Span 필터링                            | Trace 필터링                          
예시                   | model_name, temperature             | user_id, environment               

💡 권장 사용법:
  • Attributes: 함수 실행과 직접 관련된 메타데이터
  • Tags: 비즈니스 컨텍스트, 사용자 정보, 환경 정보


## 6. 여러 쿼리로 성능 비교

### 6.1 배치 실행

In [11]:
# 여러 테스트 쿼리
test_queries = [
    "RAG가 무엇인가요?",
    "Vector Store는 어떻게 작동하나요?",
    "Span의 계층 구조에 대해 설명해주세요.",
    "커스텀 Span의 장점은 무엇인가요?",
]

print("\n" + "="*80)
print("🔄 배치 실행 시작")
print("="*80)

results = []
for i, query in enumerate(test_queries, 1):
    print(f"\n[{i}/{len(test_queries)}] 처리 중...")
    result = rag_pipeline_with_custom_spans(query, top_k=3)
    results.append(result)

print(f"\n{'='*80}")
print(f"✅ 총 {len(results)}개 쿼리 처리 완료")
print("="*80)


🔄 배치 실행 시작

[1/4] 처리 중...

🔍 Query: RAG가 무엇인가요?

🌳 Root Span 생성: rag_pipeline
  ├─ 🔍 Retrieval Span 시작
  │  ✅ 3개 문서 검색 완료 (0.120초)
  ├─ 📝 Context Preparation Span 시작
  │  ✅ Context 준비 완료 (518 문자)
  ├─ 🔨 Prompt Construction Span 시작
  │  ✅ Prompt 생성 완료 (611 문자)
  └─ ✍️  Generation Span 시작
     ✅ 답변 생성 완료 (4.534초)

✅ RAG 파이프라인 완료

[2/4] 처리 중...

🔍 Query: Vector Store는 어떻게 작동하나요?

🌳 Root Span 생성: rag_pipeline
  ├─ 🔍 Retrieval Span 시작
  │  ✅ 3개 문서 검색 완료 (0.099초)
  ├─ 📝 Context Preparation Span 시작
  │  ✅ Context 준비 완료 (501 문자)
  ├─ 🔨 Prompt Construction Span 시작
  │  ✅ Prompt 생성 완료 (607 문자)
  └─ ✍️  Generation Span 시작
     ✅ 답변 생성 완료 (5.182초)

✅ RAG 파이프라인 완료

[3/4] 처리 중...

🔍 Query: Span의 계층 구조에 대해 설명해주세요.

🌳 Root Span 생성: rag_pipeline
  ├─ 🔍 Retrieval Span 시작
  │  ✅ 3개 문서 검색 완료 (0.131초)
  ├─ 📝 Context Preparation Span 시작
  │  ✅ Context 준비 완료 (518 문자)
  ├─ 🔨 Prompt Construction Span 시작
  │  ✅ Prompt 생성 완료 (623 문자)
  └─ ✍️  Generation Span 시작
     ✅ 답변 생성 완료 (5.203초)

✅ RAG 파이프라인 완료

[4/4] 처리

## 7. Trace 검색 및 분석

### 7.1 생성된 Trace 검색

In [12]:
# Trace 검색
traces = mlflow.search_traces(
    experiment_ids=[experiment.experiment_id],
    max_results=10,
    order_by=["timestamp_ms DESC"],
    return_type="list",
)

print(f"\n📊 검색된 Trace: {len(traces)}개\n")
print("="*80)
print(f"{'Request ID':<40} {'실행 시간':<15} {'Spans':<10}")
print("="*80)

for trace in traces[:5]:  # 최근 5개만 출력
    request_id = trace.info.request_id[:36]
    exec_time = f"{trace.info.execution_time_ms:.0f}ms" if trace.info.execution_time_ms else "N/A"
    num_spans = len(trace.data.spans) if trace.data.spans else 0
    
    print(f"{request_id:<40} {exec_time:<15} {num_spans:<10}")

print("="*80)


📊 검색된 Trace: 7개

Request ID                               실행 시간           Spans     
tr-49b7551534d275297ac8a68f4ba36f50      5209ms          5         
tr-21ee7b3157088706bca7d1ffdd79aa5a      5415ms          5         
tr-6fdb25a9610693b99835881f9b962696      5380ms          5         
tr-4df891d983d3a03a50721744da243ebe      4724ms          5         
tr-818d768c4e5fb30fdb7c3a21158cf117      5574ms          3         


/var/folders/v7/jvdq74bd213fzs8k9cmpl_7w0000gn/T/ipykernel_7078/3748418091.py:2: FutureWarning: Parameter 'experiment_ids' is deprecated. Please use 'locations' instead.
  traces = mlflow.search_traces(


### 7.2 Span별 성능 분석

In [13]:
def analyze_span_performance(trace):
    """
    Trace의 각 Span 성능 분석
    """
    if not trace.data.spans:
        return
    
    print("\n" + "="*80)
    print("⏱️  Span별 실행 시간 분석")
    print("="*80)
    
    span_times = []
    
    for span in trace.data.spans:
        duration_ms = (span.end_time_ns - span.start_time_ns) / 1_000_000
        span_times.append({
            "name": span.name,
            "type": span.span_type,
            "duration_ms": duration_ms
        })
    
    # 실행 시간 순 정렬
    span_times.sort(key=lambda x: x["duration_ms"], reverse=True)
    
    print(f"\n{'Span Name':<30} {'Type':<15} {'Duration (ms)':<15}")
    print("-" * 80)
    
    for span_info in span_times:
        print(f"{span_info['name']:<30} {span_info['type']:<15} {span_info['duration_ms']:>12.2f}")
    
    # 가장 느린 Span
    slowest = span_times[0]
    print("\n" + "="*80)
    print(f"🐌 가장 느린 Span: {slowest['name']} ({slowest['duration_ms']:.2f}ms)")
    print("="*80)


if traces:
    analyze_span_performance(traces[0])


⏱️  Span별 실행 시간 분석

Span Name                      Type            Duration (ms)  
--------------------------------------------------------------------------------
rag_pipeline                   CHAIN                5209.13
answer_generation              LLM                  5069.03
document_retrieval             RETRIEVER             138.13
prompt_construction            PREPROCESSING           0.68
context_preparation            PREPROCESSING           0.19

🐌 가장 느린 Span: rag_pipeline (5209.13ms)


### 7.3 성능 병목 지점 식별

In [14]:
def identify_bottlenecks(traces_list, threshold_ms=1000):
    """
    여러 Trace에서 성능 병목 지점 식별
    """
    span_stats = {}
    
    for trace in traces_list:
        if not trace.data.spans:
            continue
        
        for span in trace.data.spans:
            duration_ms = (span.end_time_ns - span.start_time_ns) / 1_000_000
            
            if span.name not in span_stats:
                span_stats[span.name] = []
            
            span_stats[span.name].append(duration_ms)
    
    print("\n" + "="*80)
    print("🔍 성능 병목 지점 분석")
    print("="*80)
    
    print(f"\n{'Span Name':<30} {'Avg (ms)':<12} {'Max (ms)':<12} {'Count':<8}")
    print("-" * 80)
    
    bottlenecks = []
    
    for span_name, durations in span_stats.items():
        avg_time = sum(durations) / len(durations)
        max_time = max(durations)
        count = len(durations)
        
        print(f"{span_name:<30} {avg_time:>10.2f} {max_time:>10.2f} {count:>6}")
        
        if avg_time > threshold_ms:
            bottlenecks.append((span_name, avg_time))
    
    if bottlenecks:
        print("\n" + "="*80)
        print(f"⚠️  병목 지점 발견 (임계값: {threshold_ms}ms):")
        for span_name, avg_time in sorted(bottlenecks, key=lambda x: x[1], reverse=True):
            print(f"  • {span_name}: {avg_time:.2f}ms (평균)")
    else:
        print(f"\n✅ 모든 Span이 임계값 {threshold_ms}ms 이내입니다.")
    
    print("="*80)


if traces:
    identify_bottlenecks(traces[:5], threshold_ms=500)


🔍 성능 병목 지점 분석

Span Name                      Avg (ms)     Max (ms)     Count   
--------------------------------------------------------------------------------
rag_pipeline                      5260.73    5574.51      5
document_retrieval                 122.29     138.13      4
context_preparation                  0.19       0.33      4
prompt_construction                  0.44       0.68      4
answer_generation                 5058.41    5281.18      4
retrieval                          103.86     103.86      1
generation                        5470.25    5470.25      1

⚠️  병목 지점 발견 (임계값: 500ms):
  • generation: 5470.25ms (평균)
  • rag_pipeline: 5260.73ms (평균)
  • answer_generation: 5058.41ms (평균)


## 8. Span 계층 구조 시각화

### 8.1 트리 형태 출력

In [15]:
def visualize_span_tree(trace):
    """
    Span 계층 구조를 트리 형태로 시각화
    """
    if not trace.data.spans:
        print("Span이 없습니다.")
        return
    
    # Parent-Child 관계 구축
    span_map = {span.span_id: span for span in trace.data.spans}
    root_spans = [span for span in trace.data.spans if span.parent_id is None]
    
    def print_span_recursive(span, level=0, is_last=False):
        # 인덴트 및 연결선
        if level == 0:
            prefix = "🌳 "
        else:
            prefix = "   " * (level - 1)
            prefix += "└─ " if is_last else "├─ "
        
        # Span 정보
        duration_ms = (span.end_time_ns - span.start_time_ns) / 1_000_000
        span_type = span.span_type if span.span_type else "UNKNOWN"
        
        print(f"{prefix}{span.name} [{span_type}] ({duration_ms:.2f}ms)")
        
        # Attributes 출력 (주요 항목만)
        if span.attributes:
            indent = "   " * level
            for key in ["llm_model", "num_docs_retrieved", "retrieval_method"]:
                if key in span.attributes:
                    print(f"{indent}   💡 {key}: {span.attributes[key]}")
        
        # Child Spans 재귀 출력
        children = [s for s in trace.data.spans if s.parent_id == span.span_id]
        for i, child in enumerate(children):
            is_last_child = (i == len(children) - 1)
            print_span_recursive(child, level + 1, is_last_child)
    
    print("\n" + "="*80)
    print("🌳 Span 계층 구조")
    print("="*80 + "\n")
    
    for root_span in root_spans:
        print_span_recursive(root_span)
    
    print("\n" + "="*80)


if traces:
    visualize_span_tree(traces[0])


🌳 Span 계층 구조

🌳 rag_pipeline [CHAIN] (5209.13ms)
├─ document_retrieval [RETRIEVER] (138.13ms)
      💡 num_docs_retrieved: 3
      💡 retrieval_method: semantic_search
├─ context_preparation [PREPROCESSING] (0.19ms)
├─ prompt_construction [PREPROCESSING] (0.68ms)
└─ answer_generation [LLM] (5069.03ms)
      💡 llm_model: global.anthropic.claude-sonnet-4-5-20250929-v1:0



## 9. 고급: Span Events 사용

### 9.1 중요 이벤트 로깅

In [21]:
def rag_pipeline_with_events(query: str) -> Dict[str, Any]:
    """
    Span Events를 사용한 RAG 파이프라인
    """
    with mlflow.start_span(name="rag_pipeline", span_type=SpanType.CHAIN) as root_span:
        root_span.set_inputs({"query": query})

        # Event: 파이프라인 시작
        # 이벤트 생성
        pipeline_started_event = SpanEvent(
            name="pipeline_started",
            timestamp=datetime.now().isoformat(), 
            attributes={"info": "pipeline 시작"}
        )
        root_span.add_event(pipeline_started_event)
        
        with mlflow.start_span(name="retrieval", span_type=SpanType.RETRIEVER) as retrieval_span:
            # Event: 검색 시작
            # 이벤트 생성
            search_initiated_event = SpanEvent(
                name="search_initiated",
                attributes={"query": query}
            )
            retrieval_span.add_event(search_initiated_event)
            
            docs = vector_store.similarity_search(query, k=3)
            
            # Event: 검색 완료
            search_completed_event = SpanEvent(
                name="search_completed",
                attributes={
                "num_docs": len(docs),
                "doc_ids": [f"doc_{i}" for i in range(len(docs))]
                }
            )
            retrieval_span.add_event(search_completed_event)
            
            retrieval_outputs = [
                Document(page_content=doc.page_content, metadata={"doc_uri": f"doc_{i}"})
                for i, doc in enumerate(docs)
            ]
            retrieval_span.set_outputs(retrieval_outputs)
        
        with mlflow.start_span(name="generation", span_type=SpanType.LLM) as gen_span:
            # Event: LLM 호출 시작
            gen_span.add_event(SpanEvent("llm_call_started", attributes={"model": AWS_MODEL_ID}))
            
            context = "\n".join([doc.page_content for doc in docs])
            
            # LLM 호출 (AWS Bedrock)
            llm = ChatBedrock(
                model_id=AWS_MODEL_ID,
                region_name=AWS_REGION,
                model_kwargs={"temperature": 0.7}
            )
            
            response = llm.invoke(f"Context: {context}\n\nQuestion: {query}")
            answer = response.content
            
            # Event: LLM 호출 완료
            gen_span.add_event(SpanEvent("llm_call_completed", attributes={"answer_length": len(answer)}))
            
            gen_span.set_outputs({"answer": answer})
        
        # Event: 파이프라인 완료
        root_span.add_event(SpanEvent("pipeline_completed", attributes={"success": True}))
        
        result = {"query": query, "answer": answer}
        root_span.set_outputs(result)
        
        return result


# Events와 함께 실행
result_with_events = rag_pipeline_with_events("Vector Store는 무엇인가요?")

print("✅ Events가 포함된 Trace 생성 완료")
print(f"💬 답변: {result_with_events['answer'][:100]}...")

✅ Events가 포함된 Trace 생성 완료
💬 답변: Vector Store는 **문서의 임베딩 벡터를 저장하고 검색하는 데이터베이스**입니다.

주요 특징은 다음과 같습니다:

## 대표적인 Vector Store
- **FAISS...


## 10. MLflow UI에서 결과 확인

### 10.1 UI 탐색 가이드

In [20]:
print("\n" + "="*80)
print("🌐 MLflow UI에서 커스텀 Span 확인하기")
print("="*80)

print("""
1️⃣ MLflow UI 실행:
   $ mlflow ui --port 5000

2️⃣ Experiment 선택:
   • 'rag_agent_custom_spans' 실험 클릭

3️⃣ Traces 탭:
   • 생성된 Trace 목록 확인
   • Request ID 클릭하여 상세 보기

4️⃣ Span 계층 구조:
   • Waterfall 차트로 시각화
   • 각 Span 클릭하여 상세 정보 확인
   • Inputs, Outputs, Attributes 확인

5️⃣ 성능 분석:
   • 실행 시간이 긴 Span 식별
   • Attributes로 설정값 확인
   • Events로 중요 이벤트 확인

6️⃣ Tags 활용:
   • Trace 레벨 태그로 필터링
   • user_id, session_id 등으로 검색
""")

print("="*80)


🌐 MLflow UI에서 커스텀 Span 확인하기

1️⃣ MLflow UI 실행:
   $ mlflow ui --port 5000

2️⃣ Experiment 선택:
   • 'rag_agent_custom_spans' 실험 클릭

3️⃣ Traces 탭:
   • 생성된 Trace 목록 확인
   • Request ID 클릭하여 상세 보기

4️⃣ Span 계층 구조:
   • Waterfall 차트로 시각화
   • 각 Span 클릭하여 상세 정보 확인
   • Inputs, Outputs, Attributes 확인

5️⃣ 성능 분석:
   • 실행 시간이 긴 Span 식별
   • Attributes로 설정값 확인
   • Events로 중요 이벤트 확인

6️⃣ Tags 활용:
   • Trace 레벨 태그로 필터링
   • user_id, session_id 등으로 검색



### 10.2 디버깅 워크플로우

In [22]:
print("\n" + "="*80)
print("🐛 커스텀 Span을 활용한 디버깅 워크플로우")
print("="*80)

print("""
📋 디버깅 단계:

1. 문제 발견:
   • MLflow UI에서 느린 Trace 식별
   • 실행 시간이 임계값을 초과하는 Trace 찾기

2. Span 분석:
   • Waterfall 차트에서 가장 시간이 오래 걸리는 Span 확인
   • 해당 Span의 Attributes 검토

3. 원인 파악:
   • Inputs/Outputs 확인으로 데이터 흐름 추적
   • Attributes에서 설정값 확인 (예: top_k, model_name)
   • Events로 중간 단계 확인

4. 가설 수립:
   • 병목 지점 특정 (예: retrieval vs generation)
   • 설정 문제 vs 데이터 문제 구분

5. 최적화:
   • 파라미터 조정 (예: top_k 감소)
   • 알고리즘 변경 (예: reranking 추가)
   • 캐싱 도입

6. 검증:
   • 수정 후 새로운 Trace 생성
   • Span 실행 시간 비교
   • 성능 개선 확인

💡 Tips:
  • Tags로 실험 버전 관리
  • Attributes로 A/B 테스트 추적
  • Events로 중요 체크포인트 기록
""")

print("="*80)


🐛 커스텀 Span을 활용한 디버깅 워크플로우

📋 디버깅 단계:

1. 문제 발견:
   • MLflow UI에서 느린 Trace 식별
   • 실행 시간이 임계값을 초과하는 Trace 찾기

2. Span 분석:
   • Waterfall 차트에서 가장 시간이 오래 걸리는 Span 확인
   • 해당 Span의 Attributes 검토

3. 원인 파악:
   • Inputs/Outputs 확인으로 데이터 흐름 추적
   • Attributes에서 설정값 확인 (예: top_k, model_name)
   • Events로 중간 단계 확인

4. 가설 수립:
   • 병목 지점 특정 (예: retrieval vs generation)
   • 설정 문제 vs 데이터 문제 구분

5. 최적화:
   • 파라미터 조정 (예: top_k 감소)
   • 알고리즘 변경 (예: reranking 추가)
   • 캐싱 도입

6. 검증:
   • 수정 후 새로운 Trace 생성
   • Span 실행 시간 비교
   • 성능 개선 확인

💡 Tips:
  • Tags로 실험 버전 관리
  • Attributes로 A/B 테스트 추적
  • Events로 중요 체크포인트 기록



## 11. 요약 및 다음 단계

### ✅ 완료한 작업
1. 커스텀 Span으로 RAG 파이프라인 세밀하게 추적
2. Span Attributes와 Trace Tags 활용
3. Span 계층 구조 설계 및 시각화
4. 성능 병목 지점 식별 및 분석
5. Span Events로 중요 이벤트 로깅
6. MLflow UI에서 디버깅 워크플로우 학습

### 📊 주요 학습 내용
- **`mlflow.start_span()`**: 수동 Span 생성 컨텍스트 매니저
- **Span Attributes**: 실행 메타데이터 (설정값, 모델명 등)
- **Trace Tags**: 비즈니스 컨텍스트 (사용자, 세션, 환경)
- **SpanType**: RETRIEVER, LLM, CHAIN 등 사전 정의 타입
- **계층 구조**: Parent-Child 관계로 복잡한 파이프라인 표현
- **성능 분석**: Span별 실행 시간으로 병목 지점 식별

### 🎯 다음 단계 (Step 4)
다음 노트북에서는 **MLflow의 Built-in Scorers**를 활용하여:
- Faithfulness, Relevance, Correctness 평가
- LLM-as-Judge 메트릭 사용
- RAG 품질 자동 측정
- 평가 결과 추적 및 비교

→ `04_mlflow_evaluation_builtin_scorers.ipynb`로 계속하세요!

### 💡 커스텀 Span의 장점
1. **세밀한 추적**: 파이프라인의 각 단계를 명확히 구분
2. **성능 최적화**: 병목 지점을 정확히 식별
3. **디버깅**: 문제 발생 위치를 빠르게 파악
4. **비즈니스 컨텍스트**: Tags로 사용자/세션 추적
5. **유연성**: Autolog로 불가능한 커스텀 로직 추적

## 📚 참고 자료

- [MLflow Tracing - Manual Instrumentation](https://mlflow.org/docs/latest/tracing/api/manual-instrumentation)
- [Span Concepts](https://mlflow.org/docs/latest/genai/concepts/span/)
- [SpanType Documentation](https://mlflow.org/docs/latest/python_api/mlflow.entities.html#mlflow.entities.SpanType)
- [Tracing Best Practices](https://mlflow.org/docs/latest/genai/tracing/)
- [OpenTelemetry Specification](https://opentelemetry.io/docs/specs/otel/)